# NHL Anytime Goal Scorer EV Calculator (FIXED)

**Goal:** Find +EV anytime goal scorer bets for today's NHL games

**What's fixed:**
- Uses decimal odds format from API (no conversion needed)
- Filters MoneyPuck to 'all' situation (no duplicates)
- Uses correct I_F_goals column
- Proper Poisson probability for anytime goal scorer
- Correct EV calculations

In [4]:
import requests
import pandas as pd
import json
from datetime import datetime
import numpy as np

In [7]:
"""
DEBUG ODDSAPI - Find out exactly what's wrong
"""

import requests
import json

def debug_oddsapi(api_key):
    """Detailed OddsAPI debugging"""
    print("=" * 60)
    print("ODDSAPI DEBUGGING")
    print("=" * 60)
    
    # Test 1: Check if NHL is available
    print("\n1️⃣ Testing sports endpoint...")
    sports_url = "https://api.the-odds-api.com/v4/sports"
    params = {'apiKey': api_key}
    
    response = requests.get(sports_url, params=params)
    print(f"Status: {response.status_code}")
    
    if response.status_code == 200:
        sports = response.json()
        nhl_sports = [s for s in sports if 'hockey' in s.get('key', '').lower() or 'nhl' in s.get('key', '').lower()]
        print(f"\n✅ NHL-related sports found:")
        for sport in nhl_sports:
            print(f"  - {sport.get('key')}: {sport.get('title')}")
            print(f"    Active: {sport.get('active')}")
            print(f"    Has outrights: {sport.get('has_outrights')}")
    else:
        print(f"❌ Failed: {response.status_code}")
        print(response.text)
        return
    
    # Test 2: Try basic NHL endpoint (no player props)
    print(f"\n2️⃣ Testing basic NHL odds (h2h)...")
    nhl_url = "https://api.the-odds-api.com/v4/sports/icehockey_nhl/odds"
    params = {
        'apiKey': api_key,
        'regions': 'us',
        'markets': 'h2h',  # Basic market
        'oddsFormat': 'decimal'
    }
    
    response = requests.get(nhl_url, params=params)
    print(f"Status: {response.status_code}")
    print(f"Requests remaining: {response.headers.get('x-requests-remaining')}")
    
    if response.status_code == 200:
        data = response.json()
        print(f"✅ Events returned: {len(data)}")
        if data:
            print(f"\nSample event:")
            print(json.dumps(data[0], indent=2)[:500])
    else:
        print(f"❌ Failed")
        print(f"Response: {response.text}")
        return
    
    # Test 3: Check available markets
    print(f"\n3️⃣ Checking available markets for NHL...")
    if data and 'bookmakers' in data[0]:
        bookmaker = data[0]['bookmakers'][0]
        print(f"\nBookmaker: {bookmaker.get('key')}")
        if 'markets' in bookmaker:
            print(f"Available market keys:")
            for market in bookmaker['markets']:
                print(f"  - {market.get('key')}")
    
    # Test 4: Try player prop markets
    print(f"\n4️⃣ Testing player prop markets...")
    
    player_markets = [
        'player_goal_scorer',
        'player_anytime_goalscorer', 
        'player_anytime_goal_scorer',
        'player_points',
        'player_shots_on_goal',
        'alternate_player_goal_scorer'
    ]
    
    for market_name in player_markets:
        params = {
            'apiKey': api_key,
            'regions': 'us',
            'markets': market_name,
            'oddsFormat': 'decimal'
        }
        
        response = requests.get(nhl_url, params=params, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            if data and 'bookmakers' in data[0] and data[0]['bookmakers']:
                if 'markets' in data[0]['bookmakers'][0] and data[0]['bookmakers'][0]['markets']:
                    print(f"✅ {market_name}: {len(data)} events")
                    # Check if has player outcomes
                    market_data = data[0]['bookmakers'][0]['markets'][0]
                    if 'outcomes' in market_data:
                        print(f"   Sample outcome: {market_data['outcomes'][0].get('description')}")
                    continue
        
        print(f"❌ {market_name}: Failed or no data")
    
    # Test 5: Check API key quota
    print(f"\n5️⃣ Checking API quota...")
    print(f"Requests remaining: {response.headers.get('x-requests-remaining', 'Unknown')}")
    print(f"Requests used: {response.headers.get('x-requests-used', 'Unknown')}")

# Run it
if __name__ == "__main__":
    API_KEY = "b81efa2d2e27c141b788a7fb02065fe0"  # UPDATE THIS
    debug_oddsapi(API_KEY)

ODDSAPI DEBUGGING

1️⃣ Testing sports endpoint...
Status: 200

✅ NHL-related sports found:
  - icehockey_ahl: AHL
    Active: True
    Has outrights: False
  - icehockey_liiga: Liiga
    Active: True
    Has outrights: False
  - icehockey_nhl: NHL
    Active: True
    Has outrights: False
  - icehockey_nhl_championship_winner: NHL Championship Winner
    Active: True
    Has outrights: True
  - icehockey_sweden_allsvenskan: HockeyAllsvenskan
    Active: True
    Has outrights: False
  - icehockey_sweden_hockey_league: SHL
    Active: True
    Has outrights: False

2️⃣ Testing basic NHL odds (h2h)...
Status: 200
Requests remaining: 408
✅ Events returned: 13

Sample event:
{
  "id": "8ce07bdd78a18c38abd68a5f3fe9203a",
  "sport_key": "icehockey_nhl",
  "sport_title": "NHL",
  "commence_time": "2025-12-23T21:00:00Z",
  "home_team": "Toronto Maple Leafs",
  "away_team": "Pittsburgh Penguins",
  "bookmakers": [
    {
      "key": "betrivers",
      "title": "BetRivers",
      "last_update": 

In [5]:
# CONFIGURATION
API_KEY = "b81efa2d2e27c141b788a7fb02065fe0"  # Replace with your actual key
BASE_URL = "https://api.the-odds-api.com/v4"

# Data paths (../ because notebook is in notebooks/ folder)
MONEYPUCK_DATA = "../data/raw/nhl_2025_26_current_season.csv"
OUTPUT_DIR = "../data/processed/"

In [6]:
"""
NHL ANYTIME GOAL SCORER EV CALCULATOR - PRODUCTION VERSION
Uses 2025-26 current season data with lineup gate
"""

# ============================================================
# CONFIGURATION
# ============================================================

CURRENT_SEASON_CSV = "nhl_2025_26_current_season.csv"
ODDS_API_KEY = "b81efa2d2e27c141b788a7fb02065fe0"  # UPDATE THIS

# ============================================================
# 1. LOAD & CLEAN CURRENT SEASON DATA
# ============================================================

def load_and_clean_data():
    """Load current season data and remove obvious errors"""
    print("=" * 60)
    print("LOADING 2025-26 SEASON DATA")
    print("=" * 60)
    
    df = pd.read_csv(CURRENT_SEASON_CSV)
    
    original_count = len(df)
    
    # Remove impossible data (>45 games, 0 games)
    df = df[(df['games_played'] > 0) & (df['games_played'] <= 45)].copy()
    
    removed = original_count - len(df)
    if removed > 0:
        print(f"⚠️ Removed {removed} players with bad data")
    
    print(f"\n✅ Loaded {len(df)} players")
    print(f"📊 Season: 2025-26")
    print(f"🏒 Teams: {df['team'].nunique()}")
    
    return df

# ============================================================
# 2. GET TODAY'S GAMES & ACTIVE LINEUPS
# ============================================================

def get_todays_active_players(season_df):
    """
    Get only players on teams playing today
    This filters out injured players like Barkov automatically
    """
    print("\n" + "=" * 60)
    print("FETCHING TODAY'S GAMES & LINEUPS")
    print("=" * 60)
    
    # Get today's schedule
    schedule_url = "https://api-web.nhle.com/v1/schedule/now"
    response = requests.get(schedule_url, timeout=15)
    
    if response.status_code != 200:
        print("❌ Could not fetch schedule")
        return None, None
    
    data = response.json()
    games_today = []
    
    if 'gameWeek' in data:
        for day in data['gameWeek']:
            if 'games' in day:
                games_today.extend(day['games'])
    
    if not games_today:
        print("⚠️ No games scheduled today")
        return None, None
    
    print(f"\n🏒 Games today: {len(games_today)}\n")
    
    # Extract game info and teams
    teams_today = set()
    game_matchups = []
    
    for game in games_today:
        away = game.get('awayTeam', {}).get('abbrev')
        home = game.get('homeTeam', {}).get('abbrev')
        
        if away and home:
            teams_today.add(away)
            teams_today.add(home)
            game_matchups.append({
                'away_team': away,
                'home_team': home,
                'game_id': game.get('id')
            })
            print(f"  {away} @ {home}")
    
    # Filter to only players on teams playing today
    todays_players = season_df[season_df['team'].isin(teams_today)].copy()
    
    # Add opponent and home/away
    team_to_info = {}
    for game in game_matchups:
        team_to_info[game['away_team']] = {
            'opponent': game['home_team'],
            'home_away': 'away'
        }
        team_to_info[game['home_team']] = {
            'opponent': game['away_team'],
            'home_away': 'home'
        }
    
    todays_players['opponent'] = todays_players['team'].map(
        lambda x: team_to_info.get(x, {}).get('opponent')
    )
    todays_players['home_away'] = todays_players['team'].map(
        lambda x: team_to_info.get(x, {}).get('home_away')
    )
    
    print(f"\n✅ {len(todays_players)} players on active rosters today")
    print(f"   (This excludes injured players like Barkov)")
    
    return todays_players, game_matchups

# ============================================================
# 3. CALCULATE GOAL PROBABILITIES
# ============================================================

def calculate_probabilities(df):
    """Calculate probability of scoring based on current season stats"""
    print("\n" + "=" * 60)
    print("CALCULATING GOAL PROBABILITIES")
    print("=" * 60)
    
    # Use the simple_xg_per_game we calculated
    df['base_prob'] = df['simple_xg_per_game']
    
    # PP1 boost (PP1 players score ~50% more often)
    pp1_boost = 1.5
    df['pp_adjusted_prob'] = df['base_prob'] * (
        1 + (df['is_pp1'] * (pp1_boost - 1))
    )
    
    # Final probability (cap at 0.35 max)
    df['goal_probability'] = df['pp_adjusted_prob'].clip(upper=0.35)
    
    print(f"\n✅ Calculated probabilities")
    print(f"\n📊 Distribution:")
    print(f"  Mean: {df['goal_probability'].mean():.3f}")
    print(f"  Median: {df['goal_probability'].median():.3f}")
    print(f"  Max: {df['goal_probability'].max():.3f}")
    
    # Show top probability players
    print(f"\n🔥 Top 10 Highest Probabilities:")
    top10 = df.nlargest(10, 'goal_probability')[
        ['name', 'team', 'goal_probability', 'goals', 'games_played', 'is_pp1']
    ]
    print(top10.to_string(index=False))
    
    return df

# ============================================================
# 4. GET ODDS FROM ODDSAPI
# ============================================================

def get_odds(api_key):
    """Fetch anytime goal scorer odds"""
    print("\n" + "=" * 60)
    print("FETCHING ODDS FROM ODDSAPI")
    print("=" * 60)
    
    url = "https://api.the-odds-api.com/v4/sports/icehockey_nhl/odds"
    
    # Try different market names
    for market in ['player_goal_scorer', 'player_anytime_goalscorer']:
        params = {
            'apiKey': api_key,
            'regions': 'us',
            'markets': market,
            'oddsFormat': 'decimal'
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            print(f"\n✅ Market '{market}' found - {len(data)} events")
            
            all_odds = []
            
            for event in data:
                if 'bookmakers' not in event:
                    continue
                
                for bookmaker in event['bookmakers']:
                    if 'markets' not in bookmaker:
                        continue
                    
                    for market_data in bookmaker['markets']:
                        if 'outcomes' not in market_data:
                            continue
                        
                        for outcome in market_data['outcomes']:
                            all_odds.append({
                                'player': outcome.get('description', '').strip(),
                                'bookmaker': bookmaker.get('key'),
                                'odds': outcome.get('price'),
                                'home_team': event.get('home_team'),
                                'away_team': event.get('away_team')
                            })
            
            if all_odds:
                df = pd.DataFrame(all_odds)
                df['implied_prob'] = 1 / df['odds']
                print(f"✅ Extracted {len(df)} player odds")
                return df
        
        print(f"⚠️ Market '{market}' failed")
    
    return None

# ============================================================
# 5. MATCH ODDS TO PLAYERS & CALCULATE EV
# ============================================================

def calculate_ev(players_df, odds_df):
    """Match players to odds and find positive EV"""
    print("\n" + "=" * 60)
    print("CALCULATING EXPECTED VALUE")
    print("=" * 60)
    
    # Normalize names
    players_df['name_norm'] = players_df['name'].str.lower().str.strip()
    odds_df['player_norm'] = odds_df['player'].str.lower().str.strip()
    
    # Merge
    merged = odds_df.merge(
        players_df[['name_norm', 'name', 'team', 'position', 
                   'goal_probability', 'goals', 'games_played', 
                   'is_pp1', 'shots', 'pp_goals']],
        left_on='player_norm',
        right_on='name_norm',
        how='inner'
    )
    
    print(f"\n✅ Matched {len(merged)} / {len(odds_df)} odds to players")
    
    if len(merged) == 0:
        print("❌ No matches - check name formatting")
        return None
    
    # Calculate EV
    merged['ev'] = (merged['goal_probability'] * merged['odds']) - 1
    merged['ev_percent'] = merged['ev'] * 100
    
    # Filter positive EV
    positive_ev = merged[merged['ev'] > 0].copy()
    positive_ev = positive_ev.sort_values('ev_percent', ascending=False)
    
    print(f"\n🎯 POSITIVE EV BETS FOUND: {len(positive_ev)}")
    
    return positive_ev

# ============================================================
# 6. MAIN PIPELINE
# ============================================================

def run_ev_calculator():
    """Run complete EV calculation"""
    print("\n" + "🏒" * 30)
    print("NHL ANYTIME GOAL SCORER EV CALCULATOR")
    print("🏒" * 30)
    print(f"\n📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")
    print(f"📊 Season: 2025-26")
    
    # Load data
    season_df = load_and_clean_data()
    
    # Get today's games
    todays_players, games = get_todays_active_players(season_df)
    
    if todays_players is None:
        print("\n❌ No games today")
        return None
    
    # Calculate probabilities
    todays_players = calculate_probabilities(todays_players)
    
    # Get odds
    odds_df = get_odds(ODDS_API_KEY)
    
    if odds_df is None:
        print("\n❌ Could not fetch odds")
        return None
    
    # Calculate EV
    results = calculate_ev(todays_players, odds_df)
    
    if results is None or len(results) == 0:
        print("\n⚠️ No positive EV found today")
        return None
    
    # Display top results
    print("\n" + "=" * 60)
    print("TOP 20 POSITIVE EV OPPORTUNITIES")
    print("=" * 60)
    
    display = results[[
        'player', 'bookmaker', 'odds', 'implied_prob',
        'goal_probability', 'ev_percent', 'team', 'is_pp1',
        'goals', 'games_played', 'pp_goals'
    ]].head(20)
    
    print(display.to_string(index=False))
    
    # Save
    results.to_csv('positive_ev_today.csv', index=False)
    print(f"\n💾 Full results saved to: positive_ev_today.csv")
    
    # Summary
    print(f"\n" + "=" * 60)
    print("SUMMARY")
    print("=" * 60)
    print(f"✅ Games analyzed: {len(games)}")
    print(f"✅ Players evaluated: {len(todays_players)}")
    print(f"✅ Positive EV bets: {len(results)}")
    print(f"📊 Avg EV: {results['ev_percent'].mean():.1f}%")
    print(f"🔥 Best EV: {results['ev_percent'].max():.1f}%")
    
    return results

# RUN IT
if __name__ == "__main__":
    results = run_ev_calculator()


🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒
NHL ANYTIME GOAL SCORER EV CALCULATOR
🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒

📅 2025-12-23 21:03
📊 Season: 2025-26
LOADING 2025-26 SEASON DATA
⚠️ Removed 2 players with bad data

✅ Loaded 737 players
📊 Season: 2025-26
🏒 Teams: 32

FETCHING TODAY'S GAMES & LINEUPS

🏒 Games today: 42

  PIT @ TOR
  DAL @ DET
  NYR @ WSH
  MTL @ BOS
  BUF @ OTT
  NJD @ NYI
  FLA @ CAR
  NSH @ MIN
  PHI @ CHI
  UTA @ COL
  CGY @ EDM
  SJS @ VGK
  SEA @ LAK
  NYR @ NYI
  BOS @ BUF
  OTT @ TOR
  TBL @ FLA
  WSH @ NJD
  DET @ CAR
  MIN @ WPG
  NSH @ STL
  CHI @ DAL
  ANA @ LAK
  EDM @ CGY
  SJS @ VAN
  COL @ VGK
  MTL @ TBL
  NYI @ CBJ
  TOR @ DET
  PIT @ CHI
  PHI @ SEA
  CBJ @ OTT
  WSH @ FLA
  NYR @ CAR
  EDM @ WPG
  BUF @ STL
  LAK @ COL
  NSH @ UTA
  BOS @ CGY
  MIN @ VGK
  SJS @ ANA
  VAN @ SEA

✅ 737 players on active rosters today
   (This excludes injured players like Barkov)

CALCULATING GOAL PROBABILITIES

✅ Calculated probabilities

📊 Distribution:
  Mean: 0.170
  Median: 0

In [9]:
"""
NHL GOAL SCORER PREDICTIONS - COMPLETE SCRIPT
Just run this entire cell - no odds needed!
"""

import pandas as pd
import requests
from datetime import datetime

# ============================================================
# CONFIGURATION
# ============================================================

CURRENT_SEASON_CSV = "nhl_2025_26_current_season.csv"

# ============================================================
# 1. LOAD DATA
# ============================================================

def load_and_clean_data():
    """Load current season data"""
    print("=" * 60)
    print("LOADING 2025-26 SEASON DATA")
    print("=" * 60)
    
    df = pd.read_csv(CURRENT_SEASON_CSV)
    df = df[(df['games_played'] > 0) & (df['games_played'] <= 45)].copy()
    
    print(f"\n✅ Loaded {len(df)} players")
    return df

# ============================================================
# 2. GET TODAY'S GAMES
# ============================================================

def get_todays_active_players(season_df):
    """Get only players on teams playing today"""
    print("\n" + "=" * 60)
    print("FETCHING TODAY'S GAMES")
    print("=" * 60)
    
    schedule_url = "https://api-web.nhle.com/v1/schedule/now"
    response = requests.get(schedule_url, timeout=15)
    
    if response.status_code != 200:
        print("❌ Could not fetch schedule")
        return None, None
    
    data = response.json()
    games_today = []
    
    if 'gameWeek' in data:
        for day in data['gameWeek']:
            if 'games' in day:
                games_today.extend(day['games'])
    
    if not games_today:
        print("⚠️ No games today")
        return None, None
    
    print(f"\n🏒 Games today: {len(games_today)}\n")
    
    teams_today = set()
    game_matchups = []
    
    for game in games_today:
        away = game.get('awayTeam', {}).get('abbrev')
        home = game.get('homeTeam', {}).get('abbrev')
        
        if away and home:
            teams_today.add(away)
            teams_today.add(home)
            game_matchups.append({'away_team': away, 'home_team': home})
            print(f"  {away} @ {home}")
    
    todays_players = season_df[season_df['team'].isin(teams_today)].copy()
    
    # Add opponent and home/away
    team_to_info = {}
    for game in game_matchups:
        team_to_info[game['away_team']] = {'opponent': game['home_team'], 'home_away': 'away'}
        team_to_info[game['home_team']] = {'opponent': game['away_team'], 'home_away': 'home'}
    
    todays_players['opponent'] = todays_players['team'].map(lambda x: team_to_info.get(x, {}).get('opponent'))
    todays_players['home_away'] = todays_players['team'].map(lambda x: team_to_info.get(x, {}).get('home_away'))
    
    print(f"\n✅ {len(todays_players)} players on active rosters")
    
    return todays_players, game_matchups

# ============================================================
# 3. CALCULATE PROBABILITIES
# ============================================================

def calculate_probabilities(df):
    """Calculate goal scoring probabilities"""
    print("\n" + "=" * 60)
    print("CALCULATING PROBABILITIES")
    print("=" * 60)
    
    df['base_prob'] = df['simple_xg_per_game']
    df['pp_adjusted_prob'] = df['base_prob'] * (1 + (df['is_pp1'] * 0.5))
    df['goal_probability'] = df['pp_adjusted_prob'].clip(upper=0.35)
    
    print(f"\n✅ Calculated probabilities")
    print(f"  Mean: {df['goal_probability'].mean():.3f}")
    print(f"  Max: {df['goal_probability'].max():.3f}")
    
    return df

# ============================================================
# 4. MAKE PREDICTIONS
# ============================================================

def run_predictions(todays_players):
    """Generate predictions - NO ODDS NEEDED"""
    print("\n" + "🏒" * 30)
    print("GOAL SCORER PREDICTIONS")
    print("🏒" * 30)
    
    # Sort by probability
    predictions = todays_players.sort_values('goal_probability', ascending=False).copy()
    
    # Add confidence tiers
    predictions['confidence'] = pd.cut(
        predictions['goal_probability'],
        bins=[0, 0.15, 0.22, 0.30, 1.0],
        labels=['Low', 'Medium', 'High', 'Very High']
    )
    
    # Display top 25 picks
    print("\n" + "=" * 80)
    print("🎯 TOP 25 GOAL SCORER PICKS")
    print("=" * 80)
    
    top_25 = predictions.head(25)
    
    display = top_25[[
        'name', 'team', 'opponent', 'home_away',
        'goal_probability', 'confidence', 'is_pp1',
        'goals', 'games_played'
    ]]
    
    print(display.to_string(index=False))
    
    # Summary stats
    print(f"\n📊 CONFIDENCE BREAKDOWN:")
    print(top_25['confidence'].value_counts().to_string())
    
    pp1_count = top_25['is_pp1'].sum()
    print(f"\n⚡ PP1 Players: {pp1_count}/25 ({pp1_count/25*100:.0f}%)")
    
    # Parlay suggestions
    print(f"\n" + "=" * 60)
    print("💎 SUGGESTED 3-LEG PARLAY (Conservative):")
    print("=" * 60)
    parlay = predictions.head(3)
    combined_prob = parlay['goal_probability'].prod()
    print(parlay[['name', 'team', 'goal_probability']].to_string(index=False))
    print(f"\nCombined probability: {combined_prob:.3f} ({combined_prob*100:.1f}%)")
    
    # Save
    predictions.to_csv('goal_scorer_predictions_today.csv', index=False)
    print(f"\n💾 Full predictions saved to: goal_scorer_predictions_today.csv")
    
    return predictions

# ============================================================
# RUN EVERYTHING
# ============================================================

if __name__ == "__main__":
    print(f"\n📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
    
    # Load data
    season_df = load_and_clean_data()
    
    # Get today's games
    todays_players, games = get_todays_active_players(season_df)
    
    if todays_players is None:
        print("\n❌ No games today")
    else:
        # Calculate probabilities
        todays_players = calculate_probabilities(todays_players)
        
        # Make predictions
        predictions = run_predictions(todays_players)
        
        print("\n✅ DONE! Check goal_scorer_predictions_today.csv for full list")


📅 2025-12-23 21:24

LOADING 2025-26 SEASON DATA

✅ Loaded 737 players

FETCHING TODAY'S GAMES

🏒 Games today: 42

  PIT @ TOR
  DAL @ DET
  NYR @ WSH
  MTL @ BOS
  BUF @ OTT
  NJD @ NYI
  FLA @ CAR
  NSH @ MIN
  PHI @ CHI
  UTA @ COL
  CGY @ EDM
  SJS @ VGK
  SEA @ LAK
  NYR @ NYI
  BOS @ BUF
  OTT @ TOR
  TBL @ FLA
  WSH @ NJD
  DET @ CAR
  MIN @ WPG
  NSH @ STL
  CHI @ DAL
  ANA @ LAK
  EDM @ CGY
  SJS @ VAN
  COL @ VGK
  MTL @ TBL
  NYI @ CBJ
  TOR @ DET
  PIT @ CHI
  PHI @ SEA
  CBJ @ OTT
  WSH @ FLA
  NYR @ CAR
  EDM @ WPG
  BUF @ STL
  LAK @ COL
  NSH @ UTA
  BOS @ CGY
  MIN @ VGK
  SJS @ ANA
  VAN @ SEA

✅ 737 players on active rosters

CALCULATING PROBABILITIES

✅ Calculated probabilities
  Mean: 0.170
  Max: 0.350

🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒
GOAL SCORER PREDICTIONS
🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒

🎯 TOP 25 GOAL SCORER PICKS
               name team opponent home_away  goal_probability confidence  is_pp1  goals  games_played
       Leo Carlsson  ANA      SJS      home      

In [10]:
"""
NHL GOAL SCORER PREDICTIONS - NO ODDS NEEDED
Pure prediction model - picks most likely scorers
"""

import pandas as pd
from datetime import datetime

def predict_goal_scorers(todays_players_df):
    """
    Rank players by goal-scoring probability
    No odds needed!
    """
    print("\n" + "=" * 60)
    print("TOP GOAL SCORER PREDICTIONS")
    print("=" * 60)
    
    # Sort by probability
    predictions = todays_players_df.sort_values('goal_probability', ascending=False).copy()
    
    # Add confidence tiers
    predictions['confidence'] = pd.cut(
        predictions['goal_probability'],
        bins=[0, 0.15, 0.22, 0.30, 1.0],
        labels=['Low', 'Medium', 'High', 'Very High']
    )
    
    return predictions

def create_betting_card(predictions, num_picks=20):
    """Create a betting card with top picks"""
    print(f"\n🎯 TOP {num_picks} GOAL SCORER PICKS")
    print("=" * 80)
    
    top_picks = predictions.head(num_picks)
    
    display = top_picks[[
        'name', 'team', 'opponent', 'home_away',
        'goal_probability', 'confidence', 'is_pp1',
        'goals', 'games_played', 'shots_per_game'
    ]]
    
    print(display.to_string(index=False))
    
    # Summary by confidence
    print(f"\n📊 PICKS BY CONFIDENCE:")
    print(top_picks['confidence'].value_counts().to_string())
    
    # PP1 breakdown
    pp1_count = top_picks['is_pp1'].sum()
    print(f"\n⚡ PP1 Players: {pp1_count}/{num_picks} ({pp1_count/num_picks*100:.0f}%)")
    
    return top_picks

def create_parlay_suggestions(predictions):
    """Suggest parlay combinations"""
    print(f"\n" + "=" * 60)
    print("SUGGESTED PARLAYS")
    print("=" * 60)
    
    # Conservative parlay (top probability guys)
    print(f"\n💎 CONSERVATIVE 3-LEG PARLAY:")
    conservative = predictions.head(3)
    combined_prob = conservative['goal_probability'].prod()
    print(conservative[['name', 'team', 'goal_probability']].to_string(index=False))
    print(f"Combined probability: {combined_prob:.3f} ({combined_prob*100:.1f}%)")
    
    # Balanced parlay (mix of tiers)
    print(f"\n⚖️ BALANCED 4-LEG PARLAY:")
    balanced = pd.concat([
        predictions.head(2),  # 2 elite
        predictions.iloc[10:12]  # 2 value
    ])
    combined_prob = balanced['goal_probability'].prod()
    print(balanced[['name', 'team', 'goal_probability']].to_string(index=False))
    print(f"Combined probability: {combined_prob:.3f} ({combined_prob*100:.1f}%)")
    
    # Lottery ticket (high upside)
    print(f"\n🎰 LOTTERY TICKET 6-LEG PARLAY:")
    lottery = predictions.iloc[5:11]  # Mid-tier guys
    combined_prob = lottery['goal_probability'].prod()
    print(lottery[['name', 'team', 'goal_probability']].to_string(index=False))
    print(f"Combined probability: {combined_prob:.3f} ({combined_prob*100:.1f}%)")

def create_game_specific_picks(predictions):
    """Best pick for each game"""
    print(f"\n" + "=" * 60)
    print("BEST SCORER PER GAME")
    print("=" * 60)
    
    # Get unique games
    predictions['game'] = predictions.apply(
        lambda x: f"{x['team']} vs {x['opponent']}" if x['home_away'] == 'home' 
        else f"{x['opponent']} vs {x['team']}", 
        axis=1
    )
    
    # Best pick per game
    best_per_game = predictions.loc[predictions.groupby('game')['goal_probability'].idxmax()]
    
    print(best_per_game[['game', 'name', 'team', 'goal_probability', 'is_pp1']].to_string(index=False))

# MAIN FUNCTION
def run_prediction_model(todays_players_df):
    """
    Complete prediction workflow - NO ODDS NEEDED
    """
    print("\n" + "🏒" * 30)
    print("NHL GOAL SCORER PREDICTIONS")
    print("🏒" * 30)
    print(f"\n📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")
    print(f"📊 Players analyzed: {len(todays_players_df)}")
    
    # Make predictions
    predictions = predict_goal_scorers(todays_players_df)
    
    # Show top picks
    top_picks = create_betting_card(predictions, num_picks=25)
    
    # Parlay suggestions
    create_parlay_suggestions(predictions)
    
    # Game-specific picks
    create_game_specific_picks(predictions)
    
    # Save
    predictions.to_csv('goal_scorer_predictions_today.csv', index=False)
    print(f"\n💾 Full predictions saved to: goal_scorer_predictions_today.csv")
    
    return predictions

# Use your existing todays_players data
if __name__ == "__main__":
    # This uses the todays_players_df you already calculated
    predictions = run_prediction_model(todays_players)
    """
QUICK START - Just predictions, no odds
Uses: nhl_2025_26_current_season.csv
"""

# Just change the final section of your existing code to:

if __name__ == "__main__":
    # Steps 1-3 (same as before)
    season_df = load_and_clean_data()
    todays_players, games = get_todays_active_players(season_df)
    todays_players = calculate_probabilities(todays_players)
    
    # NEW: Run predictions (no odds needed)
    predictions = run_prediction_model(todays_players)


🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒
NHL GOAL SCORER PREDICTIONS
🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒🏒

📅 2025-12-23 21:24
📊 Players analyzed: 737

TOP GOAL SCORER PREDICTIONS

🎯 TOP 25 GOAL SCORER PICKS
               name team opponent home_away  goal_probability confidence  is_pp1  goals  games_played  shots_per_game
       Leo Carlsson  ANA      SJS      home              0.35  Very High       1     17            36        2.472222
      Sebastian Aho  CAR      NYR      home              0.35  Very High       1     13            35        2.400000
      Brandon Hagel  TBL      MTL      home              0.35  Very High       0     18            32        2.968750
    Nikita Kucherov  TBL      MTL      home              0.35  Very High       1     13            32        3.093750
       Dylan Cozens  OTT      CBJ      home              0.35  Very High       1     11            35        2.314286
      Zach Werenski  CBJ      OTT      away              0.35  Very High       1     14            3